In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
import time
import os

In [129]:
# функция парсинга полученных данных на дни
def get_day_prognose(start, end, day, temperature, wind):
    
    today_date = datetime.now().date()
    col_name = f'Прогноз {today_date} на {datetime.now().date() + timedelta(days=day)}'
    df = pd.DataFrame({
                 "Показатели": ["Температура максимальная, по Цельсию",
                                "Температура минимальная, по Цельсию",
                                "Средняя скорость ветра, м/с"],
                 col_name: [max(temperature[start:end]),
                            min(temperature[start:end]),
                            sum(wind[start:end]) / 4]
        })
    
    return df

# функция дозаписи\сохранения в excel
def save_to_excel(df, day):
    date = datetime.now().date() + timedelta(days=day)
    try:
        # если есть отчет за прошлую дату - то мы его открываем и дописываем прогноз
        df_temp = pd.read_excel(f'reports/{date}.xlsx')
        # сохранение в excel
        df_temp.merge(df).to_excel(f"reports/{date}.xlsx", index=False)

    except:    
        # если отчета нет - просто сохраняем данные
        df.to_excel(f"reports/{date}.xlsx", index=False)

# функция расчета точности предсказаний
def calculate_precision(df, rownum):
    temp = []
    if rownum == 0:
        temp.append('Выполнение прогноза максимальной температуры, %')
    elif rownum == 1:
        temp.append('Выполнение прогноза минимальной температуры, %')
    
    for column in list(df.columns)[1:-1]:  
        temp.append(round(df.loc[rownum, column] / df.loc[rownum, 'Факт'] * 100, 2))
    
    temp.append('-') 
    df.loc[df.shape[0]] = temp        
        

try:
    os.mkdir('reports')
except:
    pass

""" 
В зависимости от времени запуска скрипта нужно выставлять значение offset равное количеству
значений температуры за сегодня (чтобы скрипт её не учитывал, а брал только значения за
последующие дни)

"""
offset = 2

if datetime.now().time().hour == 14:
    offset = 2
elif datetime.now().time().hour == 23:
    offset = 0

In [3]:
# PREDICTION REPORT (START IN 14.00)

try:
    link = "https://rp5.ru/Погода_в_Ростове-на-Дону"
    # запуск google-хрома
    browser = webdriver.Chrome('C:/chromedriver/chromedriver.exe')
    browser.get(link)

    # temperature
    temperature_elements = browser.find_elements_by_xpath("//div[@class='t_0']//b")[0+offset:12+offset]
    temperature = []
    for temp in temperature_elements:
        temperature.append(float(temp.text))

    time.sleep(15)

    # wind
    wind_elements = browser.find_elements_by_xpath("//div[@class='wv_0']")[0+offset:12+offset]
    wind = []
    for elem in wind_elements:
        wind.append(float(elem.text))

finally:
    time.sleep(5)
    browser.close()
    browser.quit()
    
    
# 1-2-3 дни начиная с текущего дня
df_today1 = get_day_prognose(0, 4, 1, temperature, wind)
df_today2 = get_day_prognose(4, 8, 2, temperature, wind)
df_today3 = get_day_prognose(8, 13, 3, temperature, wind)

# сохранение\дозапись в excel
save_to_excel(df_today1, 1)
save_to_excel(df_today2, 2)
save_to_excel(df_today3, 3)

In [7]:
# REPORT OF TARGET DAY (START IN 22.00)
try:
    link = "https://rp5.ru/Архив_погоды_в_Ростове-на-Дону"
    # запуск google-хрома
    browser = webdriver.Chrome('C:/chromedriver/chromedriver.exe')
    browser.get(link)

    # получаем температуру
    temp21 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(2) > td:nth-child(3) > div.t_0.dfs")
    temp21 = float(temp21.text)

    temp18 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(3) > td:nth-child(2) > div.t_0.dfs")
    temp18 = float(temp18.text)

    temp15 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(4) > td:nth-child(2) > div.t_0.dfs")
    temp15 = float(temp15.text)

    temp12 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(5) > td:nth-child(2) > div.t_0.dfs")
    temp12 = float(temp12.text)

    temp09 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(6) > td:nth-child(2) > div.t_0.dfs")
    temp09 = float(temp09.text)

    temp06 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(7) > td:nth-child(2) > div.t_0.dfs")
    temp06 = float(temp06.text)

    temp03 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(8) > td:nth-child(2) > div.t_0.dfs")
    temp03 = float(temp03.text)

    min_temp_arch = min([temp21, temp18, temp15, temp12, temp09, temp06, temp03])
    max_temp_arch = max([temp21, temp18, temp15, temp12, temp09, temp06, temp03])

    # пытаемся получить ветер
    wind21 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(2) > td:nth-child(9)")
    wind21 = str(wind21.text).replace(' ', '')

    wind18 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(3) > td:nth-child(8)")
    wind18 = str(wind18.text).replace(' ', '')

    wind15 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(4) > td:nth-child(8)")
    wind15 = str(wind15.text).replace(' ', '')

    wind12 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(5) > td:nth-child(8)")
    wind12 = str(wind12.text).replace(' ', '')

    wind09 = browser.find_element_by_css_selector("#archiveTable > tbody > tr:nth-child(6) > td:nth-child(8)")
    wind09 = str(wind09.text).replace(' ', '')

    # заносим ветер в список и удаляем все пустые значения
    wind_list = [wind21, wind18, wind15, wind12, wind09]
    wind_list = [x for x in wind_list if x]

    # если ветра нет - ветер 0, если был ветер (или несколько) то считаем среднее значение
    if len(wind_list) == 0:
        wind = 0
    else:
        wind_list = [float(x) for x in wind_list]
        wind = sum(wind_list) / len(wind_list)
finally:
    # ждем 5 секунд на случай если что-то не успело прогрузиться
    time.sleep(5)
    browser.close()
    browser.quit()
    

today_date = datetime.now().date()
df = pd.DataFrame({
            "Показатели": ["Температура максимальная, по Цельсию",
                           "Температура минимальная, по Цельсию",
                           "Средняя скорость ветра, м/с"],
            "Факт": [max_temp_arch,
                       min_temp_arch,
                       wind]
        })

try:
    df_temp = pd.read_excel(f'reports/{today_date}.xlsx')
    df_merged = df_temp.merge(df)
    
    calculate_precision(df_merged, 0)
    calculate_precision(df_merged, 1)
    
    df_merged.to_excel(f'reports/{today_date}.xlsx', index=False)
except:
    df.to_excel(f'reports/{today_date}.xlsx', index=False)